In [45]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 9: Sensor Boost ---</h2><p>You've just said goodbye to the rebooted rover and left Mars when you receive a faint distress signal coming from the asteroid belt.  It must be the Ceres monitoring station!</p>
<p>In order to lock on to the signal, you'll need to boost your sensors. The Elves send up the latest <em>BOOST</em> program - Basic Operation Of System Test.</p>
<p>While BOOST (your puzzle input) is capable of boosting your sensors, for <span title="Oh sure, NOW safety is a priority.">tenuous safety reasons</span>, it refuses to do so until the computer it runs on passes some checks to demonstrate it is a <em>complete Intcode computer</em>.</p>
<p><a href="5">Your existing Intcode computer</a> is missing one key feature: it needs support for parameters in <em>relative mode</em>.</p>
<p>Parameters in mode <code>2</code>, <em>relative mode</em>, behave very similarly to parameters in <em>position mode</em>: the parameter is interpreted as a position.  Like position mode, parameters in relative mode can be read from or written to.</p>
<p>The important difference is that relative mode parameters don't count from address <code>0</code>.  Instead, they count from a value called the <em>relative base</em>. The <em>relative base</em> starts at <code>0</code>.</p>
<p>The address a relative mode parameter refers to is itself <em>plus</em> the current <em>relative base</em>. When the relative base is <code>0</code>, relative mode parameters and position mode parameters with the same value refer to the same address.</p>
<p>For example, given a relative base of <code>50</code>, a relative mode parameter of <code>-7</code> refers to memory address <code>50 + -7 = <em>43</em></code>.</p>
<p>The relative base is modified with the <em>relative base offset</em> instruction:</p>
<ul>
<li>Opcode <code>9</code> <em>adjusts the relative base</em> by the value of its only parameter. The relative base increases (or decreases, if the value is negative) by the value of the parameter.</li>
</ul>
<p>For example, if the relative base is <code>2000</code>, then after the instruction <code>109,19</code>, the relative base would be <code>2019</code>. If the next instruction were <code>204,-34</code>, then the value at address <code>1985</code> would be output.</p>
<p>Your Intcode computer will also need a few other capabilities:</p>
<ul>
<li>The computer's available memory should be much larger than the initial program. Memory beyond the initial program starts with the value <code>0</code> and can be read or written like any other memory. (It is invalid to try to access memory at a negative address, though.)</li>
<li>The computer should have support for large numbers. Some instructions near the beginning of the BOOST program will verify this capability.</li>
</ul>
<p>Here are some example programs that use these features:</p>
<ul>
<li><code>109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99</code> takes no input and produces a <a href="https://en.wikipedia.org/wiki/Quine_(computing)">copy of itself</a> as output.</li>
<li><code>1102,34915192,34915192,7,4,7,99,0</code> should output a 16-digit number.</li>
<li><code>104,1125899906842624,99</code> should output the large number in the middle.</li>
</ul>
<p>The BOOST program will ask for a single input; run it in test mode by providing it the value <code>1</code>. It will perform a series of checks on each opcode, output any opcodes (and the associated parameter modes) that seem to be functioning incorrectly, and finally output a BOOST keycode.</p>
<p>Once your Intcode computer is fully functional, the BOOST program should report no malfunctioning opcodes when run in test mode; it should only output a single value, the BOOST keycode. <em>What BOOST keycode does it produce?</em></p>
</article>


In [50]:
from itertools import takewhile
from typing import Generator

from more_itertools import first


class IntcodeComputer:
    def __init__(self, s: str) -> None:
        self.state = {a: int(i) for a, i in enumerate(s.split(","))}
        self.ip = 0

    def run(self, inputs=None, reset=True) -> Generator[int, None, None]:
        inputs = inputs if inputs else []

        if reset:
            self.ip = 0

        offset = 0

        while True:
            mode_c, mode_b, mode_a, opcode = self.opcode()

            if opcode == 1:
                c = self.value(mode_c, offset)
                b = self.value(mode_b, offset)
                self.state[self.adress(mode_a, offset)] = c + b
            elif opcode == 2:
                c = self.value(mode_c, offset)
                b = self.value(mode_b, offset)
                self.state[self.adress(mode_a, offset)] = c * b
            elif opcode == 3:
                self.state[self.adress(mode_c, offset)] = inputs.pop(0)
            elif opcode == 4:
                c = self.value(mode_c, offset)
                yield c
            elif opcode == 5:
                c = self.value(mode_c, offset)
                b = self.value(mode_b, offset)
                if c != 0:
                    self.ip = b
            elif opcode == 6:
                c = self.value(mode_c, offset)
                b = self.value(mode_b, offset)
                if c == 0:
                    self.ip = b
            elif opcode == 7:
                c = self.value(mode_c, offset)
                b = self.value(mode_b, offset)
                self.state[self.adress(mode_a, offset)] = 1 if c < b else 0
            elif opcode == 8:
                c = self.value(mode_c, offset)
                b = self.value(mode_b, offset)
                self.state[self.adress(mode_a, offset)] = 1 if c == b else 0
            elif opcode == 9:
                offset += self.value(mode_c, offset)
            elif opcode == 99:
                return
            else:
                raise ValueError("1202 program alarm")

    def opcode(self):
        code = self.state[self.ip]
        self.ip += 1
        opcode = code % 100
        code //= 100
        mode_c = code % 10
        code //= 10
        mode_b = code % 10
        code //= 10
        mode_a = code % 10
        return mode_c, mode_b, mode_a, opcode

    def value(self, mode: int, offset: int) -> int:
        adress = self.adress(mode, offset)
        if adress > -1:
            if adress not in self.state:
                self.state[adress] = 0
            return self.state[adress]

        raise ValueError(
            f"Adress should be bigger then 0: adress={self.state[self.ip] + offset}"
        )

    def adress(self, mode: int, offset: int) -> int:
        ip = self.ip
        self.ip += 1

        if mode == 0:
            return self.state[ip]

        if mode == 1:
            return ip

        if mode == 2:
            return self.state[ip] + offset

        raise ValueError(f"Unknown adress mode: mode={mode}")

    def __repr__(self) -> str:
        return ",".join(str(i) for i in self.state)


tests = [
    {
        "name": "Example 1",
        "program": "109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99",
        "expected": "109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99",
    },
    {
        "name": "Example 2",
        "program": "1102,34915192,34915192,7,4,7,99,0",
        "expected": "1219070632396864",
    },
    {
        "name": "Example 3",
        "program": "104,1125899906842624,99",
        "expected": "1125899906842624",
    },
    {
        "name": "Example 4",
        "program": "1101,12,13,1985,109,2000,109,19,204,-34,99",
        "expected": "25",
    },
]


def BOOST_keycode_tester(program: str) -> str:
    icc = IntcodeComputer(program)
    return ",".join(str(i) for i in icc.run())


run_tests_params(BOOST_keycode_tester, tests)


Test Example 1 passed, for BOOST_keycode_tester.
Test Example 2 passed, for BOOST_keycode_tester.
Test Example 3 passed, for BOOST_keycode_tester.
Test Example 4 passed, for BOOST_keycode_tester.
Success


In [51]:
with open("../input/day9.txt") as f:
    puzzle = f.read()


print(f"Part I: {",".join(str(i) for i in IntcodeComputer(puzzle).run([1]))}")

Part I: 3780860499


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>3780860499</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p><em>You now have a complete Intcode computer.</em></p>
<p>Finally, you can lock on to the Ceres distress signal! You just need to boost your sensors using the BOOST program.</p>
<p>The program runs in sensor boost mode by providing the input instruction the value <code>2</code>.  Once run, it will boost the sensors automatically, but it might take a few seconds to complete the operation on slower hardware.  In sensor boost mode, the program will output a single value: <em>the coordinates of the distress signal</em>.</p>
<p>Run the BOOST program in sensor boost mode.  <em>What are the coordinates of the distress signal?</em></p>
</article>

</main>


In [52]:
print(f"Part I: {",".join(str(i) for i in IntcodeComputer(puzzle).run([2]))}")

Part I: 33343


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>33343</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
